In [1]:
from google.cloud import storage

In [2]:
project_id = "nifty-quanta-390607"
bucket_name = "awesome_mnist_bucket"
location = "us-central1"

### creating bucket

In [ ]:
storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)

In [15]:
from pathlib import Path

def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

### uploading saved model to bucket

In [30]:
upload_directory(bucket, "my_mnist_model")

### creating model registry

In [14]:
from google.cloud import aiplatform

In [15]:
server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

In [16]:
aiplatform.init(project=project_id, location=location)

In [17]:
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image
)

Creating Model
Create Model backing LRO: projects/83375718718/locations/us-central1/models/4606893247241912320/operations/5984913541155520512
Model created. Resource name: projects/83375718718/locations/us-central1/models/4606893247241912320@1
To use this Model in another session:
model = aiplatform.Model('projects/83375718718/locations/us-central1/models/4606893247241912320@1')


### creating prediction endpoint

In [18]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

Creating Endpoint
Create Endpoint backing LRO: projects/83375718718/locations/us-central1/endpoints/1922088162352431104/operations/5380023816204320768
Endpoint created. Resource name: projects/83375718718/locations/us-central1/endpoints/1922088162352431104
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/83375718718/locations/us-central1/endpoints/1922088162352431104')


### fetch model (optional)

In [4]:
from google.cloud import aiplatform

# Initialize the AI Platform with your project and location
aiplatform.init(project=project_id, location=location)

# List all models in the project
models = aiplatform.Model.list()

# Print model IDs
for model in models:
    print(f"Model Name: {model.display_name}, Model ID: {model.name}")


Model Name: mnist, Model ID: 4606893247241912320


In [9]:
model_id = "4606893247241912320"
mnist_model = aiplatform.Model(model_name=f"projects/{project_id}/locations/{location}/models/{model_id}")

### fetch endpoint (optional)

In [6]:
from google.cloud import aiplatform

# Initialize the AI Platform with your project and location
aiplatform.init(project=project_id, location=location)

# List all endpoints in the project
endpoints = aiplatform.Endpoint.list()

# Print endpoint IDs
for endpoint in endpoints:
    print(f"Endpoint Name: {endpoint.display_name}, Endpoint ID: {endpoint.name}")


Endpoint Name: mnist-endpoint, Endpoint ID: 1922088162352431104


In [7]:
endpoint_id = 1922088162352431104
endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{project_id}/locations/{location}/endpoints/{endpoint_id}")

### Deploying model to the endpoint

In [11]:
endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    # max_replica_count=5,  # The following quotas are exceeded: CustomModelServingP4GPUsPerProjectPerRegion
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_P4",
    accelerator_count=1
)

Deploying Model projects/83375718718/locations/us-central1/models/4606893247241912320 to Endpoint : projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Deploy Endpoint model backing LRO: projects/83375718718/locations/us-central1/endpoints/1922088162352431104/operations/4395916928879689728
Endpoint model deployed. Resource name: projects/83375718718/locations/us-central1/endpoints/1922088162352431104


### making predictions

In [14]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
import numpy as np
X_new = np.expand_dims(X_test[:3]/255., axis=-1)
response = endpoint.predict(instances=X_new.tolist())

In [21]:
np.round(response.predictions, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

### undeploying model and deleting endpoint

In [22]:
endpoint.undeploy_all()  # undeploy all models from the endpoint
endpoint.delete()

Undeploying Endpoint model: projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Undeploy Endpoint model backing LRO: projects/83375718718/locations/us-central1/endpoints/1922088162352431104/operations/6217622978151055360
Endpoint model undeployed. Resource name: projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Deleting Endpoint : projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Endpoint deleted. . Resource name: projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Deleting Endpoint resource: projects/83375718718/locations/us-central1/endpoints/1922088162352431104
Delete Endpoint backing LRO: projects/83375718718/locations/us-central1/operations/4114441952169033728
Endpoint resource projects/83375718718/locations/us-central1/endpoints/1922088162352431104 deleted.
